In [1]:
!pip install pandas SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 3.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from itertools import combinations
from functools import reduce


In [3]:
# Configuração do endpoint da DBpedia
sparql = SPARQLWrapper("https://dbpedia.org/sparql")
query = """
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX db: <http://dbpedia.org/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>

    SELECT DISTINCT ?anoInicio ?anoFim ?modelo ?nomeMarca ?nomeCategoria
    WHERE {
        ?c a dbo:Automobile;
           dbo:manufacturer ?marca;
           dbo:bodyStyle ?categoria;
           dbo:productionStartYear ?anoInicio;
           dbo:productionEndYear ?anoFim.

        OPTIONAL { ?c rdfs:label ?modeloEn FILTER(lang(?modeloEn) = 'en') }
        OPTIONAL { ?c rdfs:label ?modeloPt FILTER(lang(?modeloPt) = 'pt') }
        BIND(COALESCE(?modeloEn, ?modeloPt) AS ?modelo)

        OPTIONAL { ?marca rdfs:label ?marcaEn FILTER(lang(?marcaEn) = 'en') }
        OPTIONAL { ?marca rdfs:label ?marcaPt FILTER(lang(?marcaPt) = 'pt') }
        BIND(COALESCE(?marcaEn, ?marcaPt) AS ?nomeMarca)

        OPTIONAL { ?categoria rdfs:label ?categoriaEn FILTER(lang(?categoriaEn) = 'en') }
        OPTIONAL { ?categoria rdfs:label ?categoriaPt FILTER(lang(?categoriaPt) = 'pt') }
        BIND(COALESCE(?categoriaEn, ?categoriaPt) AS ?nomeCategoria)
    }
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [4]:
# Transformando os resultados em um DataFrame
data = []
for result in results["results"]["bindings"]:
    data.append({
        "Inicio": result.get("anoInicio", {}).get("value", None),
        "Fim": result.get("anoFim", {}).get("value", None),
        "Modelo": result.get("modelo", {}).get("value", None),
        "Marca": result.get("nomeMarca", {}).get("value", None),
        "Categoria": result.get("nomeCategoria", {}).get("value", None),
    })

df = pd.DataFrame(data)
df.head()

carros = df[["Modelo", "Marca", "Categoria", "Inicio", "Fim"]].dropna().to_dict(orient="records")

In [5]:
############################################################################################
################################ Regra 1 ###################################################
############################################################################################
# Carros são concorrentes se as marcas forem diferentes e as categorias forem as mesmas
def concorrentes(Modelo1=None, Modelo2=None):
    return (
        (lambda concorrentes: 
            (lambda: 
                list(map(lambda p: f"{p[0]['Modelo']} - {p[1]['Modelo']}", concorrentes))
                if not Modelo1 and not Modelo2 else
                (lambda modelos_concorrentes: 
                    list(map(lambda p: f"{p[0]['Modelo']} - {p[1]['Modelo']}", modelos_concorrentes))
                    if Modelo1 and not Modelo2 else
                    any((p[0]["Modelo"] == Modelo1 and p[1]["Modelo"] == Modelo2) or
                         (p[0]["Modelo"] == Modelo2 and p[1]["Modelo"] == Modelo1) for p in concorrentes)
                )(list(filter(lambda p: p[0]["Modelo"] == Modelo1 or p[1]["Modelo"] == Modelo1, concorrentes)))
            )() if not Modelo1 or not Modelo2 else
            any((p[0]["Modelo"] == Modelo1 and p[1]["Modelo"] == Modelo2) or
                 (p[0]["Modelo"] == Modelo2 and p[1]["Modelo"] == Modelo1) for p in concorrentes)
        )(list(filter(lambda p: p[0]["Marca"] != p[1]["Marca"] and p[0]["Categoria"] == p[1]["Categoria"], combinations(carros, 2)))))
     
############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# concorrentes()
# concorrentes("Mitsubishi Dignity")
# concorrentes("Mitsubishi Dignity", "Cadillac DTS")
# concorrentes("Cadillac DTS", "BYD M6")

In [6]:
concorrentes()

['Cadillac DTS - Praga Alfa',
 'Cadillac DTS - Rolls-Royce Phantom VI',
 'Cadillac DTS - Rolls-Royce Phantom VI',
 'Cadillac DTS - Daimler DS420',
 'Cadillac DTS - Rover Two-litre',
 'Cadillac DTS - Mitsubishi Dignity',
 'Cadillac DTS - Mitsubishi Dignity',
 'Cadillac DTS - Mitsubishi Dignity',
 'Cadillac DTS - Austin Princess',
 'Cadillac DTS - Austin Princess',
 'Cadillac DTS - Austin Sheerline',
 'Cadillac DTS - Lincoln Custom',
 'Cadillac DTS - Lincoln L series',
 'Cadillac DTS - Lincoln L series',
 'Cadillac DTS - Škoda 637',
 'Cadillac DTS - Škoda 637',
 'Cadillac DTS - Škoda VOS',
 'Cadillac DTS - ZIL-4112R',
 'Cadillac DTS - ALFA 24 HP',
 'Cadillac DTS - Alfa Romeo RM',
 'Cadillac DTS - Daimler DK400',
 'Cadillac DTS - Daimler DR450',
 'Cadillac DTS - Fiat 512',
 'Cadillac DTS - Fiat 519',
 'Cadillac DTS - Fiat 521',
 'Cadillac DTS - Fiat 524',
 'Cadillac DTS - Nissan Prince Royal',
 'Cadillac DTS - Nissan Prince Royal',
 'Cadillac DTS - Tatra 70',
 'Cadillac DTS - Tatra 77',
 

In [7]:
concorrentes("Mitsubishi Dignity")

['Cadillac DTS - Mitsubishi Dignity',
 'Cadillac DTS - Mitsubishi Dignity',
 'Cadillac DTS - Mitsubishi Dignity',
 'Cadillac V-12 - Mitsubishi Dignity',
 'Cadillac V-12 - Mitsubishi Dignity',
 'Cadillac V-12 - Mitsubishi Dignity',
 'Cadillac V-16 - Mitsubishi Dignity',
 'Cadillac V-16 - Mitsubishi Dignity',
 'Cadillac V-16 - Mitsubishi Dignity',
 'Praga Alfa - Mitsubishi Dignity',
 'Praga Alfa - Mitsubishi Dignity',
 'Praga Alfa - Mitsubishi Dignity',
 'Rolls-Royce Phantom VI - Mitsubishi Dignity',
 'Rolls-Royce Phantom VI - Mitsubishi Dignity',
 'Rolls-Royce Phantom VI - Mitsubishi Dignity',
 'Rolls-Royce Phantom VI - Mitsubishi Dignity',
 'Rolls-Royce Phantom VI - Mitsubishi Dignity',
 'Rolls-Royce Phantom VI - Mitsubishi Dignity',
 'Daimler DS420 - Mitsubishi Dignity',
 'Daimler DS420 - Mitsubishi Dignity',
 'Daimler DS420 - Mitsubishi Dignity',
 'Rover Two-litre - Mitsubishi Dignity',
 'Rover Two-litre - Mitsubishi Dignity',
 'Rover Two-litre - Mitsubishi Dignity',
 'Princess (car)

In [8]:
concorrentes("Mitsubishi Dignity", "Cadillac DTS")

True

In [9]:
concorrentes("Cadillac DTS", "BYD M6")

False

In [10]:
############################################################################################
################################ Regra 2 ###################################################
############################################################################################
# Carros de luxo se são de determinada marca ou categorias
def carro_de_luxo(Modelo=None):
    return (
        (lambda carros_luxo: 
            (lambda: 
                list(map(lambda c: c["Modelo"], carros_luxo))
                if not Modelo else
                any(c["Modelo"] == Modelo for c in carros_luxo)
            )()
        )(list(filter(lambda c: any(criterio in (c["Marca"] or "") or criterio in (c["Categoria"] or "") for criterio in ["Rolls-Royce", "Sport", "Convertible", "Limousine"]), carros)))
    )
    
############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# carro_de_luxo()
# carro_de_luxo("BYD M6")
# carro_de_luxo("Cadillac DTS")

In [11]:
carro_de_luxo()

['Cadillac DTS',
 'Rolls-Royce Corniche (2000)',
 'Rolls-Royce Phantom (seventh generation)',
 'Rolls-Royce Phantom Drophead Coupé',
 'De Tomaso Longchamp',
 'Renault Monasix',
 'Renault Primaquatre',
 'Cadillac V-12',
 'Cadillac V-16',
 'Praga Alfa',
 'Praga Alfa',
 'Rolls-Royce Dawn',
 'Rolls-Royce Phantom VI',
 'Rolls-Royce Phantom VI',
 'DeSoto Firesweep',
 'Honda City (AA)',
 'Peugeot SR1',
 'Renault Nervasport',
 'Daimler DS420',
 'Ultima Evolution',
 'Rover Two-litre',
 'Bolwell Nagari',
 'Bond Equipe',
 'Riley 12/6',
 'Rolls-Royce 100EX',
 'Ultima GTR',
 'Rolls-Royce Phantom (seventh generation)',
 'Rolls-Royce Phantom IV',
 'Rolls-Royce Silver Seraph',
 'Rolls-Royce Twenty',
 'Cadillac Provoq',
 'Saturn Outlook',
 'Honda Crossroad',
 'Hummer H2',
 'Renault Kadjar',
 'Renault Sport Spider',
 'Honda CR-V (fourth generation)',
 'Honda CR-V (third generation)',
 'Renault Egeus',
 'Hongqi LS5',
 'Huanghai Landscape F1',
 'Huanghai Landscape V3',
 'Rolls-Royce Camargue',
 'Rolls-Roy

In [12]:
carro_de_luxo("BYD M6")

False

In [13]:
carro_de_luxo("Cadillac DTS")

True

In [18]:
############################################################################################
################################ Regra 3 ###################################################
############################################################################################
# Encontrar carros que iniciem com uma letra Letra
def encontrar_carros_com_letra(Letra=None, Lista=None):
    return (
        (lambda carros_filtrados:
            (lambda: 
                list(map(lambda c: (c["Modelo"], c["Marca"]), carros_filtrados))
                if Lista is None else
                reduce(lambda acc, x: acc and (x in list(map(lambda c: (c["Modelo"], c["Marca"]), carros_filtrados))), Lista, True)
            )()
        )(
            list(filter(lambda c: c["Modelo"].startswith(Letra) if Letra else True, carros))
        )
    )

############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# encontrar_carros_com_letra()
# encontrar_carros_com_letra("X")
# encontrar_carros_com_letra("X", [("Xiali N7","FAW Group"), ("Xiali N5","FAW Group")])
# encontrar_carros_com_letra("Z", [("Xiali N7","FAW Group"), ("Xiali N5","FAW Group")])

In [19]:
encontrar_carros_com_letra()

[('Cadillac DTS', 'Cadillac'),
 ('Rolls-Royce Corniche (2000)', 'Rolls-Royce Motors'),
 ('Rolls-Royce Phantom (seventh generation)', 'Rolls-Royce Motor Cars'),
 ('Rolls-Royce Phantom Drophead Coupé', 'Rolls-Royce Motor Cars'),
 ('Saturn Relay', 'General Motors'),
 ('De Tomaso Longchamp', 'De Tomaso'),
 ('Renault Modus', 'Renault'),
 ('Renault Monasix', 'Renault'),
 ('Renault Primaquatre', 'Renault'),
 ('Renault Scénic', 'Renault'),
 ('Cadillac V-12', 'Cadillac'),
 ('Cadillac V-16', 'Cadillac'),
 ('Bedford TL', 'Bedford Vehicles'),
 ('Praga Alfa', 'Praga (company)'),
 ('Praga Alfa', 'Praga (company)'),
 ('Praga NDO', 'Praga (company)'),
 ('Rolls-Royce Dawn', 'Rolls-Royce Motor Cars'),
 ('Rolls-Royce Phantom VI', 'Rolls-Royce Motors'),
 ('Rolls-Royce Phantom VI', 'Rolls-Royce Limited'),
 ('Scania OmniCity', 'Scania AB'),
 ('DeSoto Firesweep', 'DeSoto (automobile)'),
 ('De Luxe Ford', 'Ford Motor Company'),
 ('Honda City (AA)', 'Honda'),
 ('Bertone Genesis', 'Gruppo Bertone'),
 ('Peugeot 

In [20]:
encontrar_carros_com_letra("X")

[('Xiali N7', 'FAW Group'), ('Xiali N5', 'FAW Group')]

In [21]:
encontrar_carros_com_letra("X", [("Xiali N7","FAW Group"), ("Xiali N5","FAW Group")])

True

In [22]:
encontrar_carros_com_letra("Z", [("Xiali N7","FAW Group"), ("Xiali N5","FAW Group")])

False

In [25]:
############################################################################################
################################ Regra 4 ###################################################
############################################################################################
# Encontrar carros que pertencem a uma determinada Marca
def encontrar_carros_de_marca(Marca=None, Lista=None):
    return (
        (lambda carros_filtrados:
            (lambda: 
                list(map(lambda c: (c["Modelo"], c["Marca"]), carros_filtrados))
                if Lista is None else
                reduce(lambda acc, x: acc and (x in list(map(lambda c: (c["Modelo"], c["Marca"]), carros_filtrados))), Lista, True)
            )()
        )(
            list(filter(lambda c: c["Marca"] == Marca if Marca else True, carros))
        )
    )
    
############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# encontrar_carros_de_marca()
# encontrar_carros_de_marca("ZiL")
# encontrar_carros_de_marca("ZiL", [("ZIL-118","ZiL"), ("ZIL-4102","ZiL"), ("ZIL-4102","ZiL")])
# encontrar_carros_de_marca("Fiat", [("ZIL-118","ZiL"), ("ZIL-4102","ZiL"), ("ZIL-4102","ZiL")])

In [26]:
encontrar_carros_de_marca()

[('Cadillac DTS', 'Cadillac'),
 ('Rolls-Royce Corniche (2000)', 'Rolls-Royce Motors'),
 ('Rolls-Royce Phantom (seventh generation)', 'Rolls-Royce Motor Cars'),
 ('Rolls-Royce Phantom Drophead Coupé', 'Rolls-Royce Motor Cars'),
 ('Saturn Relay', 'General Motors'),
 ('De Tomaso Longchamp', 'De Tomaso'),
 ('Renault Modus', 'Renault'),
 ('Renault Monasix', 'Renault'),
 ('Renault Primaquatre', 'Renault'),
 ('Renault Scénic', 'Renault'),
 ('Cadillac V-12', 'Cadillac'),
 ('Cadillac V-16', 'Cadillac'),
 ('Bedford TL', 'Bedford Vehicles'),
 ('Praga Alfa', 'Praga (company)'),
 ('Praga Alfa', 'Praga (company)'),
 ('Praga NDO', 'Praga (company)'),
 ('Rolls-Royce Dawn', 'Rolls-Royce Motor Cars'),
 ('Rolls-Royce Phantom VI', 'Rolls-Royce Motors'),
 ('Rolls-Royce Phantom VI', 'Rolls-Royce Limited'),
 ('Scania OmniCity', 'Scania AB'),
 ('DeSoto Firesweep', 'DeSoto (automobile)'),
 ('De Luxe Ford', 'Ford Motor Company'),
 ('Honda City (AA)', 'Honda'),
 ('Bertone Genesis', 'Gruppo Bertone'),
 ('Peugeot 

In [27]:
encontrar_carros_de_marca("ZiL")

[('ZIL-118', 'ZiL'), ('ZIL-4102', 'ZiL'), ('ZIL-4102', 'ZiL')]

In [28]:
encontrar_carros_de_marca("ZiL", [("ZIL-118","ZiL"), ("ZIL-4102","ZiL"), ("ZIL-4102","ZiL")])

True

In [29]:
encontrar_carros_de_marca("Fiat", [("ZIL-118","ZiL"), ("ZIL-4102","ZiL"), ("ZIL-4102","ZiL")])

False

In [30]:
############################################################################################
################################ Regra 5 ###################################################
############################################################################################
# Quantidade de carros que pertencem a uma determinada Marca
def carros_por_marca(Marca=None):
    return (
        (lambda quantidade: 
            print(f'Total de carros da marca {Marca}: {quantidade}' if Marca else f'Total de carros: {quantidade}')
        )(
            reduce(lambda acc, c: acc + 1 if (not Marca or c["Marca"] == Marca) else acc, carros, 0)
        )
    )
    
############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# carros_por_marca()
# carros_por_marca("Fiat")
# carros_por_marca("ZiL")

In [31]:
carros_por_marca()

Total de carros: 5164


In [32]:
carros_por_marca("Fiat")

Total de carros da marca Fiat: 130


In [33]:
carros_por_marca("ZiL")

Total de carros da marca ZiL: 3


In [34]:
############################################################################################
################################ Regra 6 ###################################################
############################################################################################
# Regra para encontrar o modelo mais antigo de uma marca ou de todos
def modelo_mais_antigo(Marca=None):
    return (
        (lambda carros_validos:
            (lambda: 
                (lambda carro_mais_antigo:
                    (lambda idade: 
                        (print(f'Modelo: {carro_mais_antigo["Modelo"]}'),
                         print(f'Marca: {carro_mais_antigo["Marca"]}'),
                         print(f'Ano Início Fabricação: {carro_mais_antigo["Inicio"]}'),
                         print(f'Idade do Modelo: {idade} anos'))
                    )(2025 - int(carro_mais_antigo["Inicio"]))
                )(sorted(carros_validos, key=lambda c: int(c["Inicio"]))[0])
            )()
        )(
            list(filter(lambda c: c.get("Inicio") and c["Inicio"].isdigit() and int(c["Inicio"]) > 1800 and (Marca is None or c["Marca"] == Marca), carros))
        ) if list(filter(lambda c: c.get("Inicio") and c["Inicio"].isdigit() and int(c["Inicio"]) > 1800 and (Marca is None or c["Marca"] == Marca), carros)) else print("Nenhum carro encontrado com os critérios informados.")
    )

############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# modelo_mais_antigo()
# modelo_mais_antigo("Fiat")
# modelo_mais_antigo("Volkswagen")

In [35]:
modelo_mais_antigo()

Modelo: Peugeot Type 3
Marca: Peugeot
Ano Início Fabricação: 1891
Idade do Modelo: 134 anos


(None, None, None, None)

In [38]:
modelo_mais_antigo("Fiat")

Modelo: Fiat 4 HP
Marca: Fiat
Ano Início Fabricação: 1899
Idade do Modelo: 126 anos


(None, None, None, None)

In [39]:
modelo_mais_antigo("Volkswagen")

Modelo: Volkswagen Beetle
Marca: Volkswagen
Ano Início Fabricação: 1938
Idade do Modelo: 87 anos


(None, None, None, None)

In [40]:
############################################################################################
################################ Regra 7 ###################################################
############################################################################################
# Pega todos os modelos que não pagam mais IPVA no Brasil por marca
def modelo_livre_de_ipva_por_marca(Marca=None, Lista=None):
    return (
        (lambda carros_isentos:
            (lambda: 
                list(map(lambda c: print((c["Modelo"], c["Marca"])), carros_isentos))
                if Lista is None else
                reduce(lambda acc, x: acc and (x in list(map(lambda c: (c["Modelo"], c["Marca"]), carros_isentos))), Lista, True)
            )()
        )(
            list(filter(lambda c: c.get("Fim") and c["Fim"].isdigit() and int(c["Fim"]) < 2025 - 15 and (Marca is None or c["Marca"] == Marca), carros))
        )
    )

############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# modelo_livre_de_ipva_por_marca()
# modelo_livre_de_ipva_por_marca("Fiat")
# modelo_livre_de_ipva_por_marca("ZiL", [('ZIL-4102', 'ZiL'), ('ZIL-118', 'ZiL'), ('ZIL-4102', 'ZiL')])
# modelo_livre_de_ipva_por_marca("Trabant", [("RGW-Auto","Trabant",1972), ("RGW-Auto","Trabant",1972), ("RGW-Auto","Trabant",1972)])

In [41]:
modelo_livre_de_ipva_por_marca()

('Rolls-Royce Corniche (2000)', 'Rolls-Royce Motors')
('Rolls-Royce Phantom (seventh generation)', 'Rolls-Royce Motor Cars')
('Saturn Relay', 'General Motors')
('De Tomaso Longchamp', 'De Tomaso')
('Renault Monasix', 'Renault')
('Renault Primaquatre', 'Renault')
('Cadillac V-12', 'Cadillac')
('Cadillac V-16', 'Cadillac')
('Bedford TL', 'Bedford Vehicles')
('Praga Alfa', 'Praga (company)')
('Praga Alfa', 'Praga (company)')
('Praga NDO', 'Praga (company)')
('Rolls-Royce Phantom VI', 'Rolls-Royce Motors')
('Rolls-Royce Phantom VI', 'Rolls-Royce Limited')
('DeSoto Firesweep', 'DeSoto (automobile)')
('De Luxe Ford', 'Ford Motor Company')
('Honda City (AA)', 'Honda')
('Bertone Genesis', 'Gruppo Bertone')
('Renault Nervasport', 'Renault')
('Daimler DS420', 'Daimler Company')
('Mercedes-Benz O305', 'Mercedes-Benz buses')
('Bedford TL', 'AWD Trucks')
('Praga NDO', 'Karosa')
('Rover Two-litre', 'Rover (car)')
('Bolwell Nagari', 'Bolwell')
('Bond Equipe', "Sharp's Commercials")
('Riley 12/6', 'Ri

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [42]:
modelo_livre_de_ipva_por_marca("Fiat")

('Fiat 850', 'Fiat')
('Fiat 1100 (1937)', 'Fiat')
('Fiat 1400 and 1900', 'Fiat')
('Fiat 1500 (1935)', 'Fiat')
('Fiat 512', 'Fiat')
('Fiat 519', 'Fiat')
('Fiat 521', 'Fiat')
('Fiat 524', 'Fiat')
('Fiat 1100 (1937)', 'Fiat')
('Fiat 128', 'Fiat')
('Fiat 126', 'Fiat')
('Fiat 147', 'Fiat')
('Fiat Bravo and Brava', 'Fiat')
('Fiat 147', 'Fiat')
('Fiat 521', 'Fiat')
('Fiat 126', 'Fiat')
('Fiat 147', 'Fiat')
('Fiat 850', 'Fiat')
('Fiat 1100', 'Fiat')
('Fiat 124', 'Fiat')
('Fiat 1100 (1937)', 'Fiat')
('Fiat 508', 'Fiat')
('Fiat 509', 'Fiat')
('Fiat 514', 'Fiat')
('Fiat 525', 'Fiat')
('Fiat 850', 'Fiat')
('Fiat 1100', 'Fiat')
('Fiat 124', 'Fiat')
('Fiat 130', 'Fiat')
('Fiat 131', 'Fiat')
('Fiat 147', 'Fiat')
('Fiat 1800 and 2100', 'Fiat')
('Fiat 2300', 'Fiat')
('Fiat 147', 'Fiat')
('Fiat 4 HP', 'Fiat')
('Fiat 850', 'Fiat')
('Fiat 124', 'Fiat')
('Fiat 128', 'Fiat')
('Fiat 130', 'Fiat')
('Fiat 131', 'Fiat')
('Fiat 850', 'Fiat')
('Fiat Bravo and Brava', 'Fiat')
('Fiat 1100 (1937)', 'Fiat')
('Fiat 11

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [43]:
modelo_livre_de_ipva_por_marca("ZiL", [('ZIL-4102', 'ZiL'), ('ZIL-118', 'ZiL'), ('ZIL-4102', 'ZiL')])

True

In [44]:
modelo_livre_de_ipva_por_marca("Trabant", [("RGW-Auto","Trabant",1972), ("RGW-Auto","Trabant",1972), ("RGW-Auto","Trabant",1972)])

False

In [45]:
############################################################################################
################################ Regra 8 ###################################################
############################################################################################
# Informa se uma marca possui ou não pelo menos um carro de luxo. Corte limita a busca a uma aparição true
def possui_carros_de_luxo(Marca=None):
    return (
        (lambda: 
            any(c["Marca"] == Marca and any(cat in c["Categoria"] for cat in ["Rolls-Royce", "Sport", "Convertible", "Limousine"]) for c in carros)
            if Marca else
            (lambda marcas_luxo: print("Marcas com carros de luxo:", list(marcas_luxo)))(set(map(lambda c: c["Marca"], filter(lambda c: any(cat in c["Categoria"] for cat in ["Rolls-Royce", "Sport", "Convertible", "Limousine"]), carros))))
        )()
    )

############################################################################################
######################## EXEMPLOS DE CONSULTAS #############################################
############################################################################################

# possui_carros_de_luxo()
# possui_carros_de_luxo("Trabant")
# possui_carros_de_luxo("SEAT")

In [46]:
possui_carros_de_luxo()

Marcas com carros de luxo: ['Rolls-Royce Motor Cars', 'Cagiva', 'Innocenti', 'Riley (automobile)', 'Rover (car)', 'SEAT', 'Auto Union', 'Sunbeam Motor Car Company', 'Spyker Cars', 'Kia Motors', 'Škoda Works', 'Land Rover', 'Audi', 'Saleen, Incorporated', 'Chrysler', 'Mazda', 'Hyundai Motor Company', 'Rolls-Royce Motors', 'JMC Yusheng', 'Great Wall Motors', 'Jaguar Cars', 'Porsche', 'Bentley Motors', 'Alpina', 'Venturi (electric car company)', 'Special Vehicle Team', 'Continental Mark II', 'Maserati', 'Daimler AG', 'BYD Auto', 'Geely Auto', 'Ford Motor Company', 'Standard-Triumph', 'Rinspeed', 'Volkswagen', 'Industrija Motornih Vozil', 'Prince Motor Company', 'Peugeot', 'Mitsubishi Motors', 'BAW', 'Valmet Automotive', 'Geely', 'Towarzystwo Budowy Samochodów AS', 'Mahindra & Mahindra', 'SsangYong Motor Company', 'Great Wall Motor', 'Plymouth (automobile)', 'American Motors', 'Monteverdi (automobile)', 'Rolls-Royce Ltd', 'Kaiser-Jeep', 'Fiat Chrysler Automobiles', 'Noble Automotive', 'ARO

In [47]:
possui_carros_de_luxo("Trabant")

False

In [48]:
possui_carros_de_luxo("SEAT")

True